In [1]:
import os
import sys
node_path = "/home/doku/.nvm/versions/node/v20.11.0/bin/node"
script_directory = os.path.abspath('')
parent_directory = os.path.dirname(script_directory)
parent_parent_directory = os.path.dirname(parent_directory)
sys.path.append(parent_parent_directory)
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import ta
import numpy as np
import datetime
import subprocess
import sqlite3
import glob
import shutil
from IPython.display import clear_output
from utilities.data_manager import ExchangeDataManager
from utilities.custom_indicators import get_n_columns, SuperTrend
from utilities.backtesting import basic_single_asset_backtest, plot_wallet_vs_asset, get_metrics, get_n_columns, plot_sharpe_evolution, plot_bar_by_month
from utilities.custom_indicators import get_n_columns
import traceback

In [9]:
from datetime import datetime, timedelta
import re

# Get the current date
today = datetime.now()

# Subtract one week from the current date
one_week_ago = today - timedelta(weeks=1)
date_one_week_ago = one_week_ago.strftime("%Y-%m-%d")

print(f"Date one week ago: {date_one_week_ago}")

original_line = 'startDate = input.time(timestamp("2023-05-01"), "Date de début", group="periode")'
date_pattern = r'\d{4}-\d{2}-\d{2}'
new_line = re.sub(date_pattern, date_one_week_ago, original_line)
print("New line:", new_line)

# The original string
original_string = """
line1
line2
startDate = input.time(timestamp("2023-05-01"), "Date de début", group="periode")
endDate = input.time(timestamp("01 Jan 2029"), "Date de fin", group="periode")
line4
"""

# Pattern to match the line starting with "startDate = input.time(timestamp("
start_date_pattern = r'^startDate = input\.time\(timestamp\("(\d{4}-\d{2}-\d{2})"\),'

# Split the original string into lines
lines = original_string.split('\n')

# Replace the line matching the pattern
for i, line in enumerate(lines):
    match = re.match(start_date_pattern, line)
    if match:
        test = match.group(0)
        print(test)
        old_date = match.group(1)
        print(old_date)
        new_line = re.sub(old_date, date_one_week_ago, line)
        lines[i] = new_line

# Join the lines back into a string
new_string = '\n'.join(lines)

print("New string:")
print(new_string)

Date one week ago: 2024-06-08
New line: startDate = input.time(timestamp("2024-06-08"), "Date de début", group="periode")
startDate = input.time(timestamp("2023-05-01"),
2023-05-01
New string:

line1
line2
startDate = input.time(timestamp("2024-06-08"), "Date de début", group="periode")
endDate = input.time(timestamp("01 Jan 2029"), "Date de fin", group="periode")
line4



In [2]:
exchange_name = "bitget"
database_directory = os.path.join(parent_parent_directory, 'database')
bitget = ExchangeDataManager(
    exchange_name=exchange_name, 
    path_download=database_directory
)
markets = bitget.exchange.load_markets()

In [6]:
print(markets)

{'TRX/USDT': {'id': 'TRXUSDT_SPBL', 'lowercaseId': None, 'symbol': 'TRX/USDT', 'base': 'TRX', 'quote': 'USDT', 'settle': None, 'baseId': 'TRX', 'quoteId': 'USDT', 'settleId': None, 'type': 'spot', 'spot': True, 'margin': False, 'swap': False, 'future': False, 'option': False, 'index': None, 'active': True, 'contract': False, 'linear': None, 'inverse': None, 'taker': 0.002, 'maker': 0.002, 'contractSize': None, 'expiry': None, 'expiryDatetime': None, 'strike': None, 'optionType': None, 'precision': {'amount': 0.0001, 'price': 1e-06, 'cost': None, 'base': None, 'quote': None}, 'limits': {'leverage': {'min': None, 'max': None}, 'amount': {'min': 0.0, 'max': 0.0}, 'price': {'min': None, 'max': None}, 'cost': {'min': 5.0, 'max': None}}, 'created': None, 'info': {'symbol': 'TRXUSDT_SPBL', 'symbolName': 'TRXUSDT', 'baseCoin': 'TRX', 'quoteCoin': 'USDT', 'minTradeAmount': '0', 'maxTradeAmount': '0', 'takerFeeRate': '0.002', 'makerFeeRate': '0.002', 'priceScale': '6', 'quantityScale': '4', 'quo

In [8]:
download_data_path_script = os.path.join(database_directory, 'download_data.js')
for market in markets:
    if 'USDT' in market and markets[market]['swap'] == True:
        print(market)
        # p = subprocess.Popen([node_path, download_data_path_script, f'{market}'], env=os.environ, stdout=subprocess.PIPE)
        # out = p.stdout.read()
        # print(out)    

BTC/USDT:USDT
ETH/USDT:USDT
XRP/USDT:USDT
EOS/USDT:USDT
BCH/USDT:USDT
LTC/USDT:USDT
ADA/USDT:USDT
ETC/USDT:USDT
LINK/USDT:USDT
TRX/USDT:USDT
DOT/USDT:USDT
DOGE/USDT:USDT
SOL/USDT:USDT
MATIC/USDT:USDT
BNB/USDT:USDT
UNI/USDT:USDT
ICP/USDT:USDT
AAVE/USDT:USDT
FIL/USDT:USDT
XLM/USDT:USDT
ATOM/USDT:USDT
XTZ/USDT:USDT
SUSHI/USDT:USDT
AXS/USDT:USDT
THETA/USDT:USDT
AVAX/USDT:USDT
SHIB/USDT:USDT
MANA/USDT:USDT
GALA/USDT:USDT
SAND/USDT:USDT
DYDX/USDT:USDT
CRV/USDT:USDT
NEAR/USDT:USDT
EGLD/USDT:USDT
KSM/USDT:USDT
AR/USDT:USDT
FTM/USDT:USDT
PEOPLE/USDT:USDT
LRC/USDT:USDT
NEO/USDT:USDT
WAVES/USDT:USDT
ALGO/USDT:USDT
IOTA/USDT:USDT
ENJ/USDT:USDT
GMT/USDT:USDT
ZIL/USDT:USDT
IOST/USDT:USDT
APE/USDT:USDT
RUNE/USDT:USDT
KNC/USDT:USDT
APT/USDT:USDT
CHZ/USDT:USDT
ROSE/USDT:USDT
ZRX/USDT:USDT
KAVA/USDT:USDT
ENS/USDT:USDT
GAL/USDT:USDT
AUDIO/USDT:USDT
SXP/USDT:USDT
C98/USDT:USDT
OP/USDT:USDT
RSR/USDT:USDT
SNX/USDT:USDT
STORJ/USDT:USDT
1INCH/USDT:USDT
COMP/USDT:USDT
IMX/USDT:USDT
LUNA2/USDT:USDT
FLOW/USDT:US